In [1]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, EsmForMaskedLM
from tokenizers import Tokenizer
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

/opt/anaconda3/envs/MachLearn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from plm_compare_progen2 import *
from protein_data import *

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model_name = "hugohrban/progen2-medium"
model, tokenizer = initialize_progen2(model_name)

Using cpu device


In [4]:
df_otc_h = pd.read_csv('/Users/johnhutchens/Desktop/Practicum/Data/DMS_ProteinGym_substitutions/OTC_HUMAN_Lo_2023.csv')

In [5]:
df_otc_h = df_otc_h.drop('DMS_score_bin', axis='columns')
df_otc_h.columns

Index(['mutant', 'mutated_sequence', 'DMS_score'], dtype='object')

In [10]:
otc_h_pg2_dict = df_otc_h.set_index(df_otc_h.columns[0]).to_dict(orient='index')

In [11]:
otc_h_pg2_dict['A102E']

{'mutated_sequence': 'MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLKNFTGEEIKYMLWLSADLKFRIKQKGEYLPLLQGKSLGMIFEKRSTRTRLSTETGFELLGGHPCFLTTQDIHLGVNESLTDTARVLSSMADAVLARVYKQSDLDTLAKEASIPIINGLSDLYHPIQILADYLTLQEHYSSLKGLTLSWIGDGNNILHSIMMSAAKFGMHLQAATPKGYEPDASVTKLAEQYAKENGTKLLLTNDPLEAAHGGNVLITDTWISMGQEEEKKKRLQAFQGYQVTMKTAKVAASDWTFLHCLPRKPEEVDDEVFYSPRSLVFPEAENRKWTIMAVMVSLLTDYSPQLQKPKF',
 'DMS_score': 0.4379999999999999}

In [ ]:
for k in otc_h_pg2_dict.keys():
    sequence = otc_h_pg2_dict[k]['mutated_sequence']
    lp, rlp, llr = collect_log_prob_pg2(sequence, model, tokenizer)

    otc_h_pg2_dict[k]['log_probs'] = lp
    otc_h_pg2_dict[k]['ref_log_probs'] = rlp
    otc_h_pg2_dict[k]['llr_matrix'] = llr


In [15]:
otc_h_pg2_dict['A102E']

{'mutated_sequence': 'MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLKNFTGEEIKYMLWLSADLKFRIKQKGEYLPLLQGKSLGMIFEKRSTRTRLSTETGFELLGGHPCFLTTQDIHLGVNESLTDTARVLSSMADAVLARVYKQSDLDTLAKEASIPIINGLSDLYHPIQILADYLTLQEHYSSLKGLTLSWIGDGNNILHSIMMSAAKFGMHLQAATPKGYEPDASVTKLAEQYAKENGTKLLLTNDPLEAAHGGNVLITDTWISMGQEEEKKKRLQAFQGYQVTMKTAKVAASDWTFLHCLPRKPEEVDDEVFYSPRSLVFPEAENRKWTIMAVMVSLLTDYSPQLQKPKF',
 'DMS_score': 0.4379999999999999,
 'log_probs': tensor([[-3.0771, -4.3744, -3.9293,  ..., -3.5056, -4.9663, -4.5736],
         [-2.4809, -4.2470, -3.2702,  ..., -3.3394, -4.9351, -4.0413],
         [-2.7221, -4.0177, -3.1940,  ..., -3.0444, -4.7724, -3.8331],
         ...,
         [-2.5317, -3.9035, -3.5848,  ..., -2.7421, -2.8584, -3.7931],
         [-2.5164, -3.9701, -2.9415,  ..., -2.8257, -4.3615, -3.7234],
         [-2.6169, -4.3407, -3.1171,  ..., -2.7873, -3.2522, -3.4560]]),
 'ref_log_probs': tensor([[-4.0998e+00],
         [-3.4723e+00],
         [-2.4184e+00],
         [-3.1936e+00],
         [-2.8248e+

In [17]:
len(otc_h_pg2_dict.keys())

1570

In [18]:
filename = '/Users/johnhutchens/Desktop/Practicum/Data/OTC_Human/pg2_otc_hum_matrices.pickle' 

with open(filename, 'wb') as f:
    pickle.dump(otc_h_pg2_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
